# 第10章 性能評価

## 10.2 評価指標を用いた自動評価

### 10.2.3 llm-jp-evalで使用される評価指標

完全一致率

In [ ]:
def calc_exact_match_ratio(trues: list[str], preds: list[str]) -> float:
    """完全一致率を算出する"""
    # どちらかの事例がなければ0
    if len(trues) == 0 or len(preds) == 0:
        return 0
    # 正解テキストと予測テキストが一致していれば1、そうでなければ0
    num_exact_match = sum(1 if t == p else 0 for t, p in zip(trues, preds))
    return num_exact_match / len(trues)

# 正解テキスト列
trues = ["entailment", "entailment", "contradiction"]
# 予測テキスト列
preds = ["entailment", "entailmen", "neutral"]
# 完全一致率を算出する
print("完全一致率:", calc_exact_match_ratio(trues, preds))

完全一致率: 0.3333333333333333


文字ベースF値

In [ ]:
from collections import Counter

def calc_char_f1(trues: list[str], preds: list[str]) -> float:
    """文字ベースF値を算出する"""
    if len(trues) == 0 or len(preds) == 0:
        return 0

    char_f1_scores = []
    for t, p in zip(trues, preds):
        # 正解テキストと予測テキストのどちらかの文字がない場合
        if len(t) == 0 or len(p) == 0:
            # 両方とも文字がないならば1、そうでなければ0
            char_f1_scores.append(float(t == p))
            break
        # 正解テキストと予測テキストの一致している文字数を算出する
        common = Counter(list(t)) & Counter(list(p))
        num_same = sum(common.values())
        # 適合率を算出する
        pre = num_same / len(p)
        # 再現率を算出する
        rec = num_same / len(t)
        # F値を算出する
        f1 = 2 * (pre * rec) / (pre + rec) if (pre + rec) != 0 else 0
        char_f1_scores.append(f1)
    return sum(char_f1_scores) / len(char_f1_scores)

# 正解の単語列
trues = ["夏目漱石"]
# 予測の単語列
preds = ["夏目 漱石"]
# 文字ベースF値を算出する
print("文字ベースF値:", calc_char_f1(trues, preds))

文字ベースF値: 0.888888888888889


集合ベースF値

In [ ]:
def calc_set_f1(trues: list[str], preds: list[str]) -> float:
    """集合ベースのF値を算出する"""
    if len(trues) == 0 or len(preds) == 0:
        return 0

    set_f1_scores = []
    # 各データで算出する
    for t, p in zip(trues, preds):
        # 正解データを行ごとに分割し、集合にする
        split_t = {x.strip() for x in t.split("\n")}
        # 予測データを行ごとに分割し、集合にする
        split_p = {x.strip() for x in p.split("\n")}
         # 適合率を算出する
        pre = sum(1 if y in split_t else 0 for y in split_p) / len(split_p)
        # 再現率を算出する
        rec = sum(1 if y in split_p else 0 for y in split_t) / len(split_t)
        # F値を算出する
        f1 = 2 * (pre * rec) / (pre + rec) if (pre + rec) != 0 else 0
        set_f1_scores.append(f1)
    return sum(set_f1_scores) / len(set_f1_scores)

# 正解集合
trues = ["営業収益 positive\n"
         "純営業収益 positive\n"
         "経常利益 positive\n"
         "当期純利益 positive"]
# 予測集合
preds = ["営業収益 positive\n"
         "純営業収益 negative\n"
         "経常利益 positive"]
# 集合ベースF値を算出する
print("集合ベースF値:", calc_set_f1(trues, preds))

集合ベースF値: 0.5714285714285715


相関係数

In [ ]:
from scipy.stats import pearsonr, spearmanr
import math

def calc_pearsonr(trues: list[str], preds: list[str]) -> float:
    """ピアソンの積率相関係数を算出する"""
    score = pearsonr(list(map(float, trues)), list(map(float, preds))).statistic
    return 0.0 if math.isnan(score) else float(score)

def calc_spearmanr(trues: list[str], preds: list[str]) -> float:
    """スピアマンの順位相関係数"""
    score = spearmanr(list(map(float, trues)), list(map(float, preds))).statistic
    return 0.0 if math.isnan(score) else float(score)

trues = ["1.2", "2.2", "3.3", "4.9"]
preds = ["1.1", "4.1", "4.0", "5.0"]
print("ピアソンの積率相関係数:", calc_pearsonr(trues, preds))
print("スピアマンの順位相関係数:", calc_spearmanr(trues, preds))

ピアソンの積率相関係数: 0.8514063149390616
スピアマンの順位相関係数: 0.7999999999999999
